In [6]:
import pandas as pd
import os
import math
from datetime import datetime
import numpy as np
from numpy import argmax

# Importing the Keras libraries and packages
# import tensorflow.keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import sys

# from imblearn.over_sampling import KMeansSMOTE, ADASYN,SMOTE
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,OneClassSVM
from time import time
from sklearn.metrics import classification_report,matthews_corrcoef
from copy import deepcopy

# path to the dataset
# file = 'Shaleeza.Dataset.v1.csv'
file = 'IoT Network Intrusion Dataset.csv'
data = pd.read_csv(file)

In [18]:
def data_preprocessing(targets_others):
    dataset = pd.read_csv(file, error_bad_lines=False, low_memory=False)
    dataset = dataset.drop(['Flow_ID', 'Src_IP', 'Dst_IP', 'Dst_Port', 'Protocol'], axis=1)
    dataset = dataset.drop(['Timestamp'], axis=1)

    # contain only single values
    dataset = dataset.drop(
        ['Fwd_PSH_Flags', 'Fwd_URG_Flags', 'Fwd_Byts/b_Avg', 'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg',
         'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Init_Fwd_Win_Byts', 'Fwd_Seg_Size_Min'], axis=1)

    dataset['Flow_Byts/s'] = round(dataset['Flow_Byts/s'],2)

    dataset = dataset.drop(targets_others, axis=1)

    dataset = dataset.reset_index()
    dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
    dataset.dropna(inplace=True)

    # correlation
    correlated_features = set()
    correlation_matrix = dataset.corr()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) >= 0.7:
                colname = correlation_matrix.columns[i]
                correlated_features.add(colname)

    dataset.drop(labels=correlated_features, axis=1, inplace=True)

    return dataset

def kFoldCV(model, data, n_fold=10):
    diff = int(len(data)/n_fold)
    results = np.zeros((1, 4))
    predictY = deepcopy(data[:,-1])
    for i in range(n_fold):
        begin = diff*i
        end = diff*(i+1)
#         if i == n_fold-1:
#             end = -1
        test = data[begin:end]
        train = deepcopy(data)
        train = np.delete(train, range(begin, end),axis=0)
        predictY[begin:end] = model.fit(train[:,:-1], train[:,-1].astype("int")).predict(test[:,:-1])
    t = classification_report(data[:,-1].astype("int"), predictY.astype("int"))
    print(t)
    print(matthews_corrcoef(data[:,-1].astype("int"), predictY.astype("int")))
        
#         results = results + getResults(model, train[:,:-1], train[:,-1],test[:,:-1],test[:,-1])
#     return results/n_fold

def getResults(model, X_train, y_train,X_test,y):
    predictY = model.fit(X_train, y_train).predict(X_test)
    t = classification_report(y, predictY)#, target_names=['0', '1', '2']
    return t


In [11]:
data = data_preprocessing(['Label', 'Cat'])
data = data.values
data = data[:,1:]
X_train, y_train = data[:,:-1], data[:,-1]
y_train = LabelEncoder().fit_transform(y_train)
data = np.c_[X_train,y_train.astype('int')]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12196\1158925801.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataset = pd.read_csv(file, error_bad_lines=False, low_memory=False)


In [17]:
data[:,-1].astype("int")

array([2, 0, 8, ..., 8, 6, 5])

In [5]:
t = time()
length = [3, 4, 5]
for i in length:
    model = DecisionTreeClassifier(max_depth=i)
    kFoldCV(model, data)


print(time()-t)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      0.99      1.00     59391
    MITM ARP Spoofing       0.00      0.00      0.00     35377
    Mirai-Ackflooding       0.00      0.00      0.00     55124
  Mirai-HTTP Flooding       0.78      0.00      0.00     55818
Mirai-Hostbruteforceg       0.38      0.99      0.55    121178
   Mirai-UDP Flooding       0.61      0.83      0.71    183189
               Normal       0.50      0.00      0.00     40073
        Scan Hostport       0.00      0.00      0.00     22192
         Scan Port OS       1.00      0.00      0.00     53073

             accuracy                           0.53    625415
            macro avg       0.47      0.31      0.25    625415
         weighted avg       0.53      0.53      0.41    625415



C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      0.99      1.00     59391
    MITM ARP Spoofing       0.00      0.00      0.00     35377
    Mirai-Ackflooding       0.00      0.00      0.00     55124
  Mirai-HTTP Flooding       0.97      0.00      0.00     55818
Mirai-Hostbruteforceg       0.38      0.99      0.55    121178
   Mirai-UDP Flooding       0.72      0.82      0.77    183189
               Normal       0.84      0.80      0.81     40073
        Scan Hostport       0.00      0.00      0.00     22192
         Scan Port OS       1.00      0.00      0.00     53073

             accuracy                           0.58    625415
            macro avg       0.54      0.40      0.35    625415
         weighted avg       0.60      0.58      0.48    625415



C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      0.99      1.00     59391
    MITM ARP Spoofing       0.00      0.00      0.00     35377
    Mirai-Ackflooding       0.00      0.00      0.00     55124
  Mirai-HTTP Flooding       1.00      0.00      0.00     55818
Mirai-Hostbruteforceg       0.70      0.59      0.64    121178
   Mirai-UDP Flooding       0.58      0.97      0.72    183189
               Normal       0.84      0.80      0.81     40073
        Scan Hostport       0.00      0.00      0.00     22192
         Scan Port OS       0.43      0.94      0.59     53073

             accuracy                           0.62    625415
            macro avg       0.50      0.48      0.42    625415
         weighted avg       0.58      0.62      0.53    625415

296.8145170211792


C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
t = time()
length = [5]
for i in length:
    model = DecisionTreeClassifier(max_depth=i)
    kFoldCV(model, data)


print(time()-t)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00     59391
           1       0.00      0.00      0.00     35377
           2       0.00      0.00      0.00     55124
           3       0.97      0.00      0.00     55818
           4       0.70      0.59      0.64    121178
           5       0.58      0.97      0.72    183189
           6       0.84      0.80      0.81     40073
           7       0.00      0.00      0.00     22192
           8       0.43      0.94      0.59     53073

    accuracy                           0.62    625415
   macro avg       0.50      0.48      0.42    625415
weighted avg       0.58      0.62      0.53    625415

0.5548899201404829
45.45096778869629


In [6]:
t = time()
hidden_layer_sizes  = [100, 200, 300]
max_iter = [100, 200, 300]
for i in hidden_layer_sizes:
    for j in max_iter:
        model = MLPClassifier(hidden_layer_sizes=i, max_iter=j)
        kFoldCV(model, data)


print(time()-t)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00     59391
    MITM ARP Spoofing       0.93      0.06      0.12     35377
    Mirai-Ackflooding       0.43      0.00      0.00     55124
  Mirai-HTTP Flooding       0.17      0.00      0.00     55818
Mirai-Hostbruteforceg       0.48      0.00      0.00    121178
   Mirai-UDP Flooding       0.33      1.00      0.49    183189
               Normal       0.97      0.16      0.28     40073
        Scan Hostport       0.00      0.00      0.00     22192
         Scan Port OS       1.00      0.00      0.00     53073

             accuracy                           0.40    625415
            macro avg       0.59      0.25      0.21    625415
         weighted avg       0.54      0.40      0.26    625415



C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00     59391
    MITM ARP Spoofing       0.93      0.02      0.03     35377
    Mirai-Ackflooding       0.41      0.00      0.00     55124
  Mirai-HTTP Flooding       0.27      0.00      0.00     55818
Mirai-Hostbruteforceg       0.25      0.00      0.00    121178
   Mirai-UDP Flooding       0.33      1.00      0.49    183189
               Normal       0.96      0.17      0.29     40073
        Scan Hostport       0.00      0.00      0.00     22192
         Scan Port OS       1.00      0.00      0.00     53073

             accuracy                           0.40    625415
            macro avg       0.57      0.24      0.20    625415
         weighted avg       0.50      0.40      0.26    625415



C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

In [20]:
t = time()
hidden_layer_sizes  = [300]
max_iter = [200]
for i in hidden_layer_sizes:
    for j in max_iter:
        model = MLPClassifier(hidden_layer_sizes=i, max_iter=j)
        kFoldCV(model, data)


print(time()-t)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59391
           1       0.93      0.11      0.20     35377
           2       0.41      0.01      0.01     55124
           3       0.28      0.00      0.00     55818
           4       0.13      0.00      0.00    121178
           5       0.33      1.00      0.50    183189
           6       0.90      0.18      0.30     40073
           7       0.00      0.00      0.00     22192
           8       1.00      0.00      0.00     53073

    accuracy                           0.41    625415
   macro avg       0.55      0.25      0.22    625415
weighted avg       0.47      0.41      0.27    625415

0.32424166478113636
9756.128085136414


In [ ]:
t = time()
Cs = [10, 100, 1000]
gammas = [0.01, 0.1,1]
for i in Cs:
    for j in gammas:
        model = SVC(kernel = 'rbf', C = i, gamma = j)
        kFoldCV(model, data)

print(time()-t)


In [21]:
n_estimators  = [10, 100, 200]
max_depth = [3, 4, 5]
n_estimators  = [10]
max_depth = [5]
for i in n_estimators:
    for j in max_depth:
        model = RandomForestClassifier(n_estimators=i, max_depth=j)
        kFoldCV(model, data)

print(time()-t)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59391
           1       0.91      0.16      0.27     35377
           2       0.31      0.00      0.01     55124
           3       0.36      0.08      0.13     55818
           4       0.44      0.92      0.59    121178
           5       0.67      0.89      0.77    183189
           6       0.83      0.82      0.82     40073
           7       1.00      0.00      0.00     22192
           8       0.76      0.09      0.15     53073

    accuracy                           0.61    625415
   macro avg       0.70      0.44      0.42    625415
weighted avg       0.64      0.61      0.53    625415

0.5377341624147388
9957.252769947052


In [22]:
t = time()
n_estimators  = [10, 100, 200]
max_depth = [3, 4, 5]
n_estimators  = [200]
max_depth = [5]
data2 = np.c_[X_train,y_train.reshape(len(X_train),1)]
for i in n_estimators:
    for j in max_depth:
        model = XGBClassifier(n_estimators=i, max_depth=j,objective='mlogloss')
        kFoldCV(model, data2)


print(time()-t)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59391
           1       0.97      0.97      0.97     35377
           2       0.27      0.25      0.26     55124
           3       0.27      0.24      0.26     55818
           4       0.86      0.95      0.90    121178
           5       0.78      0.78      0.78    183189
           6       0.94      0.92      0.93     40073
           7       0.95      0.43      0.59     22192
           8       0.78      0.89      0.83     53073

    accuracy                           0.76    625415
   macro avg       0.76      0.72      0.72    625415
weighted avg       0.75      0.76      0.75    625415

0.7083410467604206
4974.656225681305


In [24]:

    
def kFoldCV(model, data, n_fold=10):
    diff = int(len(data)/n_fold)
    results = np.zeros((1, 4))
    predictY = deepcopy(data[:,-1]).reshape(len(data[:,-1]),1)
    for i in range(n_fold):
        begin = diff*i
        end = diff*(i+1)
#         if i == n_fold-1:
#             end = -1
        test = data[begin:end]
        train = deepcopy(data)
        train = np.delete(train, range(begin, end),axis=0)
#         print(train[:,-1].shape)
#         y_train = y_train.astype('int').reshape(len(X_train),1)
        model.fit(train[:,:-1], train[:,-1].astype(int))
        predictY[begin:end] = model.predict(test[:,:-1])
#         predictY[begin:end] = model.fit(train[:,:-1], train[:,-1].astype("int")).predict(test[:,:-1])
    t = classification_report(data[:,-1].astype("int"), predictY.astype("int"))
    print(t)
    print(matthews_corrcoef(data[:,-1].astype("int"), predictY.astype("int")))

In [25]:
t = time()
n_estimators  = [10, 100, 200]
max_depth = [3, 4, 5]
n_estimators  = [100]
max_depth = [5]
for i in n_estimators:
    for j in max_depth:
        model = CatBoostClassifier(n_estimators=i, max_depth=j)
        kFoldCV(model, data)

print(time()-t)

Learning rate set to 0.5
0:	learn: 1.1403856	total: 806ms	remaining: 1m 19s
1:	learn: 0.9400992	total: 1.77s	remaining: 1m 26s
2:	learn: 0.8505618	total: 2.43s	remaining: 1m 18s
3:	learn: 0.7919946	total: 3.42s	remaining: 1m 22s
4:	learn: 0.7389716	total: 4.14s	remaining: 1m 18s
5:	learn: 0.7090442	total: 4.92s	remaining: 1m 17s
6:	learn: 0.6774051	total: 5.63s	remaining: 1m 14s
7:	learn: 0.6607679	total: 6.97s	remaining: 1m 20s
8:	learn: 0.6477620	total: 7.81s	remaining: 1m 18s
9:	learn: 0.6368383	total: 8.65s	remaining: 1m 17s
10:	learn: 0.6234161	total: 9.55s	remaining: 1m 17s
11:	learn: 0.6125255	total: 10.5s	remaining: 1m 16s
12:	learn: 0.6019413	total: 11.3s	remaining: 1m 15s
13:	learn: 0.5953322	total: 12s	remaining: 1m 13s
14:	learn: 0.5884765	total: 12.7s	remaining: 1m 11s
15:	learn: 0.5811984	total: 13.6s	remaining: 1m 11s
16:	learn: 0.5771360	total: 14.3s	remaining: 1m 9s
17:	learn: 0.5740382	total: 15.1s	remaining: 1m 8s
18:	learn: 0.5684895	total: 15.9s	remaining: 1m 7s
19

61:	learn: 0.4933795	total: 44.7s	remaining: 27.4s
62:	learn: 0.4926277	total: 45.3s	remaining: 26.6s
63:	learn: 0.4918374	total: 46.1s	remaining: 25.9s
64:	learn: 0.4910354	total: 46.8s	remaining: 25.2s
65:	learn: 0.4901089	total: 47.6s	remaining: 24.5s
66:	learn: 0.4896755	total: 48.2s	remaining: 23.8s
67:	learn: 0.4892155	total: 49s	remaining: 23s
68:	learn: 0.4886373	total: 49.5s	remaining: 22.3s
69:	learn: 0.4881926	total: 50.4s	remaining: 21.6s
70:	learn: 0.4872690	total: 51.1s	remaining: 20.9s
71:	learn: 0.4869088	total: 51.7s	remaining: 20.1s
72:	learn: 0.4856628	total: 52.4s	remaining: 19.4s
73:	learn: 0.4846317	total: 53s	remaining: 18.6s
74:	learn: 0.4842282	total: 54s	remaining: 18s
75:	learn: 0.4839019	total: 54.6s	remaining: 17.2s
76:	learn: 0.4834836	total: 55.4s	remaining: 16.6s
77:	learn: 0.4827806	total: 56s	remaining: 15.8s
78:	learn: 0.4820943	total: 56.9s	remaining: 15.1s
79:	learn: 0.4809836	total: 57.6s	remaining: 14.4s
80:	learn: 0.4804894	total: 58.3s	remaining

22:	learn: 0.5537615	total: 41.7s	remaining: 2m 19s
23:	learn: 0.5508072	total: 43.8s	remaining: 2m 18s
24:	learn: 0.5452670	total: 45.9s	remaining: 2m 17s
25:	learn: 0.5426265	total: 47.7s	remaining: 2m 15s
26:	learn: 0.5394691	total: 49.5s	remaining: 2m 13s
27:	learn: 0.5364240	total: 51.5s	remaining: 2m 12s
28:	learn: 0.5335675	total: 53.2s	remaining: 2m 10s
29:	learn: 0.5309292	total: 54.9s	remaining: 2m 8s
30:	learn: 0.5292350	total: 55.7s	remaining: 2m 4s
31:	learn: 0.5274612	total: 56.5s	remaining: 2m
32:	learn: 0.5251595	total: 57.6s	remaining: 1m 56s
33:	learn: 0.5227175	total: 58.5s	remaining: 1m 53s
34:	learn: 0.5217445	total: 59.5s	remaining: 1m 50s
35:	learn: 0.5205854	total: 1m	remaining: 1m 47s
36:	learn: 0.5191277	total: 1m 1s	remaining: 1m 44s
37:	learn: 0.5174816	total: 1m 2s	remaining: 1m 41s
38:	learn: 0.5162896	total: 1m 3s	remaining: 1m 38s
39:	learn: 0.5147757	total: 1m 4s	remaining: 1m 36s
40:	learn: 0.5136810	total: 1m 4s	remaining: 1m 33s
41:	learn: 0.5125654	

82:	learn: 0.4788260	total: 57.7s	remaining: 11.8s
83:	learn: 0.4784469	total: 58.4s	remaining: 11.1s
84:	learn: 0.4771447	total: 59s	remaining: 10.4s
85:	learn: 0.4768608	total: 59.7s	remaining: 9.72s
86:	learn: 0.4766708	total: 1m	remaining: 9.02s
87:	learn: 0.4762835	total: 1m 1s	remaining: 8.33s
88:	learn: 0.4761158	total: 1m 1s	remaining: 7.63s
89:	learn: 0.4759260	total: 1m 2s	remaining: 6.92s
90:	learn: 0.4754734	total: 1m 2s	remaining: 6.23s
91:	learn: 0.4750323	total: 1m 3s	remaining: 5.54s
92:	learn: 0.4745155	total: 1m 4s	remaining: 4.84s
93:	learn: 0.4734722	total: 1m 4s	remaining: 4.15s
94:	learn: 0.4732893	total: 1m 5s	remaining: 3.46s
95:	learn: 0.4728274	total: 1m 6s	remaining: 2.77s
96:	learn: 0.4725392	total: 1m 7s	remaining: 2.08s
97:	learn: 0.4723653	total: 1m 7s	remaining: 1.38s
98:	learn: 0.4719617	total: 1m 8s	remaining: 693ms
99:	learn: 0.4716664	total: 1m 9s	remaining: 0us
Learning rate set to 0.5
0:	learn: 1.1361321	total: 524ms	remaining: 51.8s
1:	learn: 0.93

43:	learn: 0.5123869	total: 33.3s	remaining: 42.3s
44:	learn: 0.5110434	total: 33.9s	remaining: 41.5s
45:	learn: 0.5088755	total: 34.6s	remaining: 40.7s
46:	learn: 0.5071592	total: 35.2s	remaining: 39.7s
47:	learn: 0.5055232	total: 36.1s	remaining: 39.1s
48:	learn: 0.5045725	total: 36.7s	remaining: 38.2s
49:	learn: 0.5034279	total: 37.6s	remaining: 37.6s
50:	learn: 0.5022284	total: 38.4s	remaining: 36.9s
51:	learn: 0.5011869	total: 38.9s	remaining: 35.9s
52:	learn: 0.5005159	total: 39.4s	remaining: 35s
53:	learn: 0.4995722	total: 40.3s	remaining: 34.4s
54:	learn: 0.4984017	total: 41s	remaining: 33.5s
55:	learn: 0.4965462	total: 41.8s	remaining: 32.8s
56:	learn: 0.4953497	total: 42.4s	remaining: 32s
57:	learn: 0.4946998	total: 43.2s	remaining: 31.3s
58:	learn: 0.4941787	total: 43.9s	remaining: 30.5s
59:	learn: 0.4931330	total: 44.4s	remaining: 29.6s
60:	learn: 0.4926481	total: 45.1s	remaining: 28.8s
61:	learn: 0.4916239	total: 45.9s	remaining: 28.1s
62:	learn: 0.4908678	total: 46.5s	rem

2:	learn: 0.8522639	total: 916ms	remaining: 29.6s
3:	learn: 0.7945443	total: 1.22s	remaining: 29.3s
4:	learn: 0.7419039	total: 1.58s	remaining: 30s
5:	learn: 0.7078282	total: 1.86s	remaining: 29.2s
6:	learn: 0.6813845	total: 2.14s	remaining: 28.4s
7:	learn: 0.6656922	total: 2.43s	remaining: 28s
8:	learn: 0.6511131	total: 2.73s	remaining: 27.6s
9:	learn: 0.6378250	total: 3.02s	remaining: 27.2s
10:	learn: 0.6231792	total: 3.38s	remaining: 27.3s
11:	learn: 0.6129948	total: 3.71s	remaining: 27.2s
12:	learn: 0.6022347	total: 4.03s	remaining: 27s
13:	learn: 0.5968353	total: 4.46s	remaining: 27.4s
14:	learn: 0.5911212	total: 4.96s	remaining: 28.1s
15:	learn: 0.5823449	total: 5.72s	remaining: 30s
16:	learn: 0.5762719	total: 6.5s	remaining: 31.7s
17:	learn: 0.5720838	total: 7.26s	remaining: 33.1s
18:	learn: 0.5685481	total: 7.97s	remaining: 34s
19:	learn: 0.5620934	total: 8.68s	remaining: 34.7s
20:	learn: 0.5561072	total: 9.31s	remaining: 35s
21:	learn: 0.5526769	total: 10.1s	remaining: 35.8s
2

65:	learn: 0.4867510	total: 46.9s	remaining: 24.2s
66:	learn: 0.4862576	total: 47.6s	remaining: 23.4s
67:	learn: 0.4859128	total: 48.3s	remaining: 22.7s
68:	learn: 0.4849024	total: 48.9s	remaining: 22s
69:	learn: 0.4837661	total: 49.7s	remaining: 21.3s
70:	learn: 0.4834304	total: 50.3s	remaining: 20.6s
71:	learn: 0.4822224	total: 51s	remaining: 19.8s
72:	learn: 0.4812042	total: 51.7s	remaining: 19.1s
73:	learn: 0.4808684	total: 52.3s	remaining: 18.4s
74:	learn: 0.4805057	total: 53.1s	remaining: 17.7s
75:	learn: 0.4798248	total: 53.6s	remaining: 16.9s
76:	learn: 0.4793724	total: 54.3s	remaining: 16.2s
77:	learn: 0.4791029	total: 54.8s	remaining: 15.4s
78:	learn: 0.4788689	total: 55.5s	remaining: 14.8s
79:	learn: 0.4778340	total: 56.4s	remaining: 14.1s
80:	learn: 0.4772938	total: 57s	remaining: 13.4s
81:	learn: 0.4765367	total: 57.7s	remaining: 12.7s
82:	learn: 0.4761563	total: 58.3s	remaining: 11.9s
83:	learn: 0.4756982	total: 58.9s	remaining: 11.2s
84:	learn: 0.4755012	total: 59.7s	rem